# Equivalent du lambda Main

In [1]:
# Load all the python files modified not to use AWS : main process
%run Code/classification/loadAndCleanData.py
%run Code/Utils/utilNormalizer.py
%run Code/classification/utilCsv.py
%run Code/classification/processTextData.py
%run Code/classification/tfidfClassification.py
%run Code/classification/loadPurcheaseData.py

INTO LOAD DATA
INTO UTILS
INTO PROCESS DATA
INTO LOAD DATA
INTO PROCESS DATA
INTO TFIDF CLASSIF
INTO UTILS
INTO TFIDF CLASSIF
INTO load purchease


In [2]:
# Load all the python files modified not to use AWS : raw input process
%run Code/raw_treatments/googlePlaces.py
%run Code/raw_treatments/rawTreatment.py
%run Code/raw_treatments/EnrichLocation.py

INTO GOOGLE PLACE
INTO GOOGLE PLACE
INTO RAW TREATMENT
INTO RAW TREATMENT
INTO ENRICH LOCATION PLACE


In [3]:
# %load Code/lambdas/lambdaMain.py
import json
import warnings
warnings.filterwarnings("ignore")
import pprint

print "INTO LAMBDA"

# Load python files
p = ProcessText()
l = LoadCleanData()
u = Normalizer()
e = EnrichLocation()


# Load data for test
event_raw = json.load(open("data/smart-tickets-rawpayload-example.json", "rb"))
event_structured = json.load(open("data/smart-tickets-structuredpayload-example.json", "rb"))


def eventHandler(event):
    """
    Main lambda for our classification.
    It invokes lambda enrich location as a first step.
    Case RAW : only add lcoation and google place type if finded
    Case Structured : send Purchease classification, our classif is only used if Purchease send Non-Reconnu
    NOTHING IS SAVED INTO DB IN THIS VERSION
    :param event: json input send by Purchease
    :param context: non need
    :return: output json to be send to Purchease
    """
    pp = pprint.PrettyPrinter(indent=4)
    print "event init :", pp.pprint(event)
    # Process for the raw inputs
    event_enrich = e.enrich_location(event)
    
    new_event_smartt = event_enrich['smartticket']

    if new_event_smartt['extraction_type'] == 'STRUCTURED':
        ll = LoadPurchease(new_event_smartt)
        output_full = ll.fill_input_with_classif(new_event_smartt)
        # Clean output for send into post : del purchease_cat and rmw_cat, only stay cat
        return respond('', ll.good_shape_output_for_post(output_full))
    else:
        return respond('', event_enrich)


def respond(err, res=None):
    return {
        'statusCode': '404' if err else '200',
        'body': json.dumps(err) if err else res,
        'headers': {
            'Content-Type': 'application/json',
        },
    }


INTO LAMBDA


In [4]:
# Structured process
eventHandler(event_structured)

event init :{   u'date': u'17-06-2016 20:33',
    u'extraction_type': u'STRUCTURED',
    u'light_image_url': u'http://receipts.fidmarques.com/receipts/production/21/2016/12/21/20161221-1541-3390-64161-1/20161221-1541-3390-64161-1_prerotated.jpg',
    u'lines': [   {   u'category_image_url': u'http://cdn1.skerou.com/images/categories/0ea39cf20fcc51d0f2b78a910cdf3ad761f5bfee.png',
                      u'category_name': u'Boissons',
                      u'ocr_processed_description': u'LIMONADE CRF',
                      u'ocr_raw_description': u'LIMONADE CRF',
                      u'quantity': 1,
                      u'total_price': 0.75,
                      u'unit_price': 0.75},
                  {   u'category_image_url': u'http://cdn1.skerou.com/images/categories/629efb6f620e7e7a9ba4aa074973a593a2469d43.png',
                      u'category_name': u'\xc9picerie Sal\xe9e',
                      u'ocr_processed_description': u'1/2 OLIVE FARCIES',
                      u'ocr_raw_d

{'body': '{"smartticket": {"status": "finished", "extraction_type": "STRUCTURED", "store_address": {"latitude": 48.847611, "city": "Market Vincennes", "street": "1 Place J Spire-lemaitre", "longitude": 2.452771, "zip_code": "94300"}, "uuid": "20161221-1541-3390-64161", "total": 84.19, "retailer_name": "Carrefour", "lines": [{"total_price": 0.75, "ocr_processed_description": "LIMONADE CRF", "unit_price": 0.75, "ocr_raw_description": "LIMONADE CRF", "category_image_url": "http://cdn1.skerou.com/images/categories/0ea39cf20fcc51d0f2b78a910cdf3ad761f5bfee.png", "quantity": 1, "category_name": "Boissons"}, {"total_price": 2.14, "ocr_processed_description": "1/2 OLIVE FARCIES", "unit_price": 1.07, "ocr_raw_description": "1/2 OLIVE FARCIES", "category_image_url": "http://cdn1.skerou.com/images/categories/629efb6f620e7e7a9ba4aa074973a593a2469d43.png", "quantity": 2, "category_name": "\\u00c9picerie Sal\\u00e9e"}, {"total_price": 3.84, "ocr_processed_description": "1/4 THON NATREL PN", "unit_pri

In [5]:
# Raw process
eventHandler(event_raw)

event init :{   u'date': u'20-07-2016 18:32',
    u'extraction_type': u'RAW',
    u'light_image_url': u'http://receipts.fidmarques.com/receipts/production/21/2016/12/09/20161209-1558-2474-62401-1/20161209-1558-2474-62401-1_prerotated.jpg',
    u'lines': [   {   u'category_image_url': u'http://cdn1.skerou.com/images/products/skerou_created/unknown_product.png',
                      u'category_name': u'NON RECONNU',
                      u'ocr_processed_description': u'Date: 20/07/2016 18:32 Nb Article: 1',
                      u'ocr_raw_description': None,
                      u'quantity': None,
                      u'total_price': None,
                      u'unit_price': None},
                  {   u'category_image_url': u'http://cdn1.skerou.com/images/products/skerou_created/unknown_product.png',
                      u'category_name': u'NON RECONNU',
                      u'ocr_processed_description': u'Caisse: 367 Ticket: 6202208',
                      u'ocr_raw_description'

{'body': {'analytics_result': 'SUCCESS',
  'smartticket': {u'date': u'20-07-2016 18:32',
   u'extraction_type': u'RAW',
   u'light_image_url': u'http://receipts.fidmarques.com/receipts/production/21/2016/12/09/20161209-1558-2474-62401-1/20161209-1558-2474-62401-1_prerotated.jpg',
   u'lines': [{'category_image_url': u'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png',
     'category_name': 'Mode et Beaut\xc3\xa9',
     'ocr_processed_description': 'Mode et Beaut\xc3\xa9',
     'ocr_raw_description': '',
     'quantity': 1,
     'total_price': 2.1,
     'unit_price': 2.1}],
   u'nb_products': 0,
   u'nb_recognized_products': 0,
   u'retailer_image_url': u'http://cdn1.skerou.com/images/retailers/inconnu.png',
   u'retailer_name': u'Galeries Lafayette Haussmann',
   u'status': u'finished',
   u'store_address': {u'city': u'Paris',
    u'latitude': 48.8736449,
    u'longitude': 2.3322383,
    u'street': u'Boulevard Haussmann',
    'street_number': u'40',
    u'zip_code': u'

# Model training from Purchease outputs - Pickle creation

In [6]:
import pandas as pd
import re
import sys  

# reload(sys)  
# sys.setdefaultencoding('latin1')
import boto3
table = boto3.resource('dynamodb').Table('prod-analytics-smarttickets')

In [7]:
# Import data from dynamoDB of Amazon
u = Normalizer()
filepath = 'data/results_stocked.csv'

tickets = table.scan()
if 'Items' in tickets:
    dict_db = u.replace_decimals(tickets['Items'])
    data_df = []
    for p in dict_db:
        for line in p['lines']:
            if line['category_name_purchease'] != 'NON RECONNU':
                data_df.append([line['category_name_purchease'], line['category_name_rmw'],
                                line['ocr_processed_description']])
    with open(filepath, 'wb') as myfile:
        UnicodeWriter(myfile).writerows(data_df)

    print "CSV has been saved into results_stocked.csv"

CSV has been saved into results_stocked.csv


In [8]:
# Import data 
df_1_raw = pd.read_csv("data/results_stocked.csv", sep=",", encoding="utf-8", names=['purchease_cat','rmw_cat','description'])
df_1 = df_1_raw.drop_duplicates()
print df_1_raw.shape
print df_1.shape


(2367, 3)
(569, 3)


In [9]:
# Pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

# Parameters
params_grid = {'tfidf__min_df': [0, 0.0005, 0.001],'tfidf__max_df': [0.5,0.7]
               , 'tfidf__ngram_range': [(1, 2),(1, 1)],
               'rf__min_samples_leaf': [1, 3, 5],
               }
# TF-IDF
vectorizer = TfidfVectorizer(sublinear_tf=True)
# RF
rf = RandomForestClassifier(n_estimators=10, random_state=123)
# Pipeline
pipe = Pipeline([('tfidf', vectorizer), ('rf', rf)])
# GridSearch
model = GridSearchCV(pipe, param_grid=params_grid, n_jobs=-1, cv=5, verbose=1, refit=True)

# Cleaning inputs
X_df = [re.compile('[^\D]').sub('',x.lower()) for x in df_1['description']]
y = df_1['purchease_cat']
result_model = model.fit(X_df, y)
result_model
print result_model.score(X_df, y)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.9s


0.93848857645


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.5s finished


In [10]:
# Pickling
import cPickle as pickle
with open("models/dumpPipeline.pkl", "wb") as file1:
    pickle.dump(result_model, file1)
print ("Pipeline saved")

Pipeline saved
